In [1]:
from src.utils import extract_feature_from_image, postprocessing
from ultralytics import YOLO
from pycaret.classification import load_model, predict_model
import easyocr
import pandas as pd

In [2]:
# define image and model path
image_dir = "src/data/07-2021 - 07-2022/"
filename = "1.jpg"
box_model_path = "models/yolov8_box.pt"
symbol_model_path = "models/yolov8_symbol.pt"
classifier_path = "models/extract_model"

# load model
box_model = YOLO(box_model_path)
symbol_model = YOLO(symbol_model_path)
classifier = load_model(classifier_path)
reader = easyocr.Reader(["en"], gpu=True)

Transformation Pipeline and Model Successfully Loaded


In [3]:
# extract data from image
extract_rt_df, extract_lt_df = extract_feature_from_image(
    filename, image_dir, box_model, symbol_model, reader
)

# post processing right and left data
feature_rt_df = postprocessing(extract_rt_df)
feature_lt_df = postprocessing(extract_lt_df)


0: 640x448 1 ear_graph, 1 pta_table, 22.3ms
Speed: 4.0ms preprocess, 22.3ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 Air Rt Unmaskeds, 8 Air Lt Unmaskeds, 1 Air Rt Masked, 4 Bone Lt Unmaskeds, 4 Bone Rt Maskeds, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


In [4]:
# classified degree of hearing loss
predict_right_hearing_loss = predict_model(classifier, data=feature_rt_df)
predict_left_hearing_loss = predict_model(classifier, data=feature_lt_df)

# concat right and left data and rename index
result = pd.concat([predict_right_hearing_loss, predict_left_hearing_loss], axis=0)
result.index = ['right', 'left']

# map prediction label
label = ['Normal', 'Mild', 'Moderate', 'Moderately Severe', 'Severe', 'Profound']
result['prediction_label'] = result['prediction_label'].apply(lambda x: label[x])

result

,pta_ac,pta_bc,sl,srt,pb,250_ac,500_ac,1000_ac,2000_ac,4000_ac,6000_ac,8000_ac,500_bc,1000_bc,2000_bc,4000_bc,prediction_label,prediction_score
right,59.648117,59.077152,57.038578,55.0,7724.0,30.949831,36.480026,40.999069,37.272583,41.745735,41.534515,55.976303,35.880524,40.834648,35.495209,41.767498,Moderately Severe,1.0000
left,38.250561,37.403461,30.000000,40.0,400.0,76.224075,60.896267,56.343590,61.704502,55.982513,66.017326,71.039986,60.046516,56.159088,61.025860,55.819618,Mild,0.9999
